In [1]:
import json
import base64
import requests
import tensorflow as tf
import pandas as pd

In [3]:
# Load the dataset (adjust the path if needed)
heartatt = pd.read_csv('data/heart.csv')
heartatt.head(5)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,hrtatt
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [9]:
# Sample data for testing
test_data = {
    "age": 55,
    "sex": 1,  # Male
    "cp": 1,  # Chest Pain type: atypical angina
    "trtbps": 140,
    "chol": 250,
    "fbs": 1,
    "restecg": 1,
    "thalachh": 150,
    "exng": 1,
    "oldpeak": 1.4, 
    "slp": 2,
    "caa": 2,
    "thall": 2
}

In [10]:
# Function to prepare JSON payload
def prepare_json(test_data):
    feature_spec = {}

    for key, value in test_data.items():
        if isinstance(value, float):
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    b64_encoded_proto = base64.b64encode(example_proto).decode()

    json_payload = json.dumps({
        "signature_name": "serving_default",
        "instances": [{"examples": {"b64": b64_encoded_proto}}]
    })

    return json_payload

In [11]:
# Prepare the JSON payload
payload = prepare_json(test_data)

# Model endpoint URL
model_url = "https://heart-attack-prediction-production.up.railway.app/v1/models/heart-attack-prediction-model:predict"

# Make a prediction request
try:
    response = requests.post(model_url, data=payload, headers={"Content-Type": "application/json"})
    response.raise_for_status()  # Raise an exception for any HTTP error

    predictions = response.json().get("predictions")
    if predictions:
        prediction_value = predictions[0][0]
        result = "High chance of heart attack" if prediction_value > 0.5 else "Low chance of heart attack"
    else:
        result = "Error: No predictions found in response."

    print(f"Input Data:\n{test_data}\n")
    print(f"Prediction Result: {result}")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    print(f"Response content: {response.content}")  # Print response content for debugging
except Exception as err:
    print(f"An error occurred: {err}")

Input Data:
{'age': 55, 'sex': 1, 'cp': 1, 'trtbps': 140, 'chol': 250, 'fbs': 1, 'restecg': 1, 'thalachh': 150, 'exng': 1, 'oldpeak': 1.4, 'slp': 2, 'caa': 2, 'thall': 2}

Prediction Result: High chance of heart attack
